In [ ]:
import numpy as np
import pickle as pkl
import time
from datetime import timedelta

from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from utils import PianoDataset
from model import LSTMModel

In [ ]:
with open('processed_data.pkl', 'rb') as f:
    (normalized_sequences, encoded_labels, vocab, d_min, d_max) = pkl.load(f)

In [ ]:
# load dataset
dataset = PianoDataset(normalized_sequences, encoded_labels)

batch_size = 32
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# create training/val subsets, DataLoader objects
train_dataset, test_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for x_notes, x_durations, y_notes, y_durations in train_loader:
    print("X Notes shape:", x_notes.shape)       # Should be (batch_size, 32)
    print("X Durations shape:", x_durations.shape)  # Should be (batch_size, 32)
    print("Y Notes shape:", y_notes.shape)     # Should be (batch_size,)
    print("Y Durations shape:", y_durations.shape)     # Should be (batch_size,)
    break

In [ ]:
def model_test(test_data_x, test_data_y, model):
	#tensor_test_x = torch.IntTensor(test_data_x)
	#tensor_test_y = torch.LongTensor(test_data_y)
	test_outputs = model(test_data_x)	
	test_labels = test_data_y
	
	predict_label = torch.argmax(test_outputs, dim=1)	
	
	acc = (predict_label == test_labels).sum() / float(len(test_labels))
	return float(acc)

def note_accuracy(outputs, labels):
	predict_label = torch.argmax(outputs, dim=1)	
	
	acc = (predict_label == labels).sum() / float(len(labels))
	return float(acc)

def duration_accuracy(outputs, labels):
	pass

def evaluate(model, test_loader):
    model.eval()  # Set model to evaluation mode
    total_notes = 0
    correct_notes = 0
    total_duration_error = 0
    num_samples = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for x_notes, x_durations, y_notes, y_durations in test_loader:
            # Move data to the appropriate device (GPU or CPU)
            #x_notes, x_durations = x_notes.to(device), x_durations.to(device)
            #y_notes, y_durations = y_notes.to(device), y_durations.to(device)

            # Forward pass
            note_pred, duration_pred = model(x_notes, x_durations)

            # Notes: Compare predicted class with ground truth
            predicted_notes = torch.argmax(note_pred, dim=1)  # Get index of highest logit
            correct_notes += (predicted_notes == y_notes).sum().item()
            total_notes += y_notes.size(0)

            # Durations: Compute regression error
            total_duration_error += F.l1_loss(duration_pred.squeeze(), y_durations, reduction='sum').item()

            num_samples += y_durations.size(0)

    # Calculate metrics
    note_accuracy = correct_notes / total_notes
    avg_duration_error = total_duration_error / num_samples

    return note_accuracy, avg_duration_error


In [ ]:
# Model parameters
vocab_size = 1840
embedding_dim = 64
hidden_dim1 = 256
hidden_dim2 = 128
model = LSTMModel(vocab_size, embedding_dim, hidden_dim1, hidden_dim2)

In [ ]:
#model.load_state_dict(torch.load("model_parameters.pth"))

In [ ]:
note_acc, dur_err = evaluate(model, test_loader)
print(f'{note_acc:.4f}')
print(f'{dur_err:.4f}')
results = {0: [None, note_acc, dur_err]}

In [ ]:
# Model Params
num_epochs = 1
alpha = 0.1
criterion_note = nn.CrossEntropyLoss()
criterion_duration = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for x_notes, x_durations, y_note, y_duration in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        note_pred, duration_pred = model(x_notes, x_durations)
        
        # Compute combined loss
        loss_note = criterion_note(note_pred, y_note)
        loss_duration = criterion_duration(duration_pred.squeeze(), y_duration)
        loss = loss_note + alpha*loss_duration 
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    note_acc, dur_err = evaluate(model, test_loader)
    elapsed = time.time() - start
    t = str(timedelta(seconds=elapsed))
    print(f"Epoch {epoch + 1}: Time - {t}, Loss - {running_loss:.6f}, Note Acc - {note_acc:.4f}, Dur Err - {dur_err:.4f}")
    results[epoch+1] = [running_loss, note_acc, dur_err]
    torch.save(model.state_dict(), f'models/{epoch+1}_parameters.pth')